In [ ]:
import pandas as pd
strava_df = pd.read_csv('reviews_strava.csv')

In [ ]:
strava_df = strava_df \
    .assign(has_leaderboard = lambda d: d['review'].str.contains('leaderboard', case=False))
strava_df

In [ ]:
strava_df['has_leaderboard'].sum()
f= strava_df['has_leaderboard']
strava_df_leaderboard = strava_df[f]

In [ ]:

strava_df_leaderboard

In [ ]:

from transformers import pipeline
#import tensorflow
import torch
# Allocate a pipeline for sentiment-analysis
# classifier = pipeline('sentiment-analysis')
# strava_df['sentiment'] = classifier(strava_df['review'])
# strava_df


In [ ]:
import flair
from flair.models import TextClassifier
from flair.data import Sentence

sia = TextClassifier.load('en-sentiment')

def flair_prediction(x):
    sentence = Sentence(x)
    sia.predict(sentence)
    score = sentence.labels[0]
    #print(score)
    if "POSITIVE" in str(score):
        return "pos"
    elif "NEGATIVE" in str(score):
        return "neg"
    else:
        return "neu"
    
def flair_prediction_value(x):
    sentence = Sentence(x)
    sia.predict(sentence)
    score = sentence.labels[0]
    return score.score

In [ ]:
greetings = "Hello World"

print((greetings))

In [ ]:
#strava_df_leaderboard
strava_df_leaderboard["flair_sentiment"] = strava_df_leaderboard["review"].apply(flair_prediction)

#strava_df_leaderboard["sentences"] = strava_df_leaderboard["review"].apply(tokenize.sent_tokenize)
strava_df_leaderboard["word_count"] = strava_df_leaderboard['review'].str.split().str.len()
# strava_df_leaderboard["sentiment_score"] = strava_df_leaderboard["sentences"].apply(lambda x: [flair_prediction(y) for y in x])




In [ ]:
import time
start = time.time()
strava_df_leaderboard.loc[:,"flair_sentiment"] = strava_df_leaderboard["review"].apply(flair_prediction)
end = time.time()
print("Time to compute using flair on leaderboard was")
print(end-start)

In [ ]:
df_grouped = strava_df_leaderboard.groupby(["rating", "flair_sentiment"]).sum()
df_grouped

# Try to do analysis using VADER

In [ ]:
import nltk
import time
nltk.download('vader_lexicon')
import numpy as np
from nltk.sentiment import SentimentIntensityAnalyzer
import operator

sia = SentimentIntensityAnalyzer()

#df["sentiment"] = df["reviews.text"].apply(lambda x: max({k: sia.polarity_scores(x)[k] for k in good_keys}.items(), key=operator.itemgetter(1))[0])
df = strava_df_leaderboard
df.loc[:, "word_count"] = df["review"].str.split().str.len()


In [ ]:
%%time
1+1

In [ ]:
df

In [ ]:
#%%time


df.loc[:, "vader_sentiment_score"] = df.loc[:, "review"].apply(lambda x: sia.polarity_scores(x)["compound"])

df.loc[:, "vader_sentiment"] = np.select([df["vader_sentiment_score"] < 0, df["vader_sentiment_score"] == 0, df["vader_sentiment_score"] > 0], ['neg', 'neu', 'pos'])


In [ ]:
# rating_sorted_dfs = []
# for i in range(1, 5):
#     rating_sorted_dfs.append([df[df["rating"] == i]])

# for rating_df in rating_sorted_dfs:
#     print(rating_df[0].describe())

df.loc[:,'count'] = 1
df_grouped = df.groupby(["rating", "vader_sentiment"]).sum()

df.to_csv("strava_lb_sentiments.csv")

df_grouped